In [15]:
import numpy
import pandas as pd
import gumpy
import parse

pd.options.display.max_columns=999

In [2]:
reference = gumpy.Genome("../gumpy/config/NC_000962.3.gbk.gz", show_progress_bar=True)

100%|██████████| 100/100 [00:01<00:00, 72.57it/s]


In [4]:
def parse_who_catalog(filename):
    '''Parses the WHO TB catalog

    Args:
        filename (str): Path to the WHO catalog
    Returns:
        pd.dataframe: Dataframe containing the mutations
    '''
    df = pd.read_excel(filename, sheet_name="Genome_indices")
    return df

In [5]:
#Load the catalogue
data = parse_who_catalog("WHO-UCN-GTB-PCI-2021.7-eng.xlsx")

#The catalogue is grouped by gene, so we can store gene masks until they are no longer required
masks = {}

#Setup details for drugs
drug_columns = [
    'RIF_Conf_Grade',
    'INH_Conf_Grade',
    'EMB_Conf_Grade',
    'PZA_Conf_Grade',
    'LEV_Conf_Grade',
    'MXF_Conf_Grade',
    'BDQ_Conf_Grade',
    'LZD_Conf_Grade',
    'CFZ_Conf_Grade',
    'DLM_Conf_Grade',
    'AMI_Conf_Grade',
    'STM_Conf_Grade',
    'ETH_Conf_Grade',
    'KAN_Conf_Grade',
    'CAP_Conf_Grade']
drugs = {
    drug.split("_")[0]: {
        "R": set(),
        "U": set(),
        "S": set(),
        # "F": set()
        } for drug in drug_columns}
genes = set()

In [19]:
data[:5]

,gene_name,gene_locus,variant,codon_number,genome_index,ref_nt,alt_nt,ref_aa,alt_aa,Total_R,Total_S,RIF_R,RIF_S,RIF_Conf_Grade,INH_R,INH_S,INH_Conf_Grade,EMB_R,EMB_S,EMB_Conf_Grade,PZA_R,PZA_S,PZA_Conf_Grade,LEV_R,LEV_S,LEV_Conf_Grade,MXF_R,MXF_S,MXF_Conf_Grade,BDQ_R,BDQ_S,BDQ_Conf_Grade,LZD_R,LZD_S,LZD_Conf_Grade,CFZ_R,CFZ_S,CFZ_Conf_Grade,DLM_R,DLM_S,DLM_Conf_Grade,AMI_R,AMI_S,AMI_Conf_Grade,STM_R,STM_S,STM_Conf_Grade,ETH_R,ETH_S,ETH_Conf_Grade,KAN_R,KAN_S,KAN_Conf_Grade,CAP_R,CAP_S,CAP_Conf_Grade,final_annotation.Reference,final_annotation.Position,final_annotation.ReferenceNucleotide,final_annotation.AlternativeNucleotide,final_annotation.Gene,final_annotation.Effect,final_annotation.LocusTag,final_annotation.TentativeHGVSNucleotidicAnnotation,final_annotation.ProteinId,final_annotation.TentativeHGVSProteicAnnotation
0,gyrB,Rv0005,gyrB_c-170t,NaN,5070,c,t,NaN,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3) Uncertain significance,0.0,1.0,3) Uncertain significance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC_000962.3,5070,c,t,gyrB,upstream_gene_variant,Rv0005,c.-170C>T,NaN,NaN
1,gyrB,Rv0005,gyrB_-165_del_1_cg_c,NaN,5075,cg,c,NaN,NaN,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,3) Uncertain significance,0.0,1.0,3) Uncertain significance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC_000962.3,5075,cg,c,gyrB,upstream_gene_variant,Rv0005,c.-164delG,NaN,NaN
2,gyrB,Rv0005,gyrB_-165_ins_2_cg_cgtc,NaN,5075,cg,cgtc,NaN,NaN,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,3) Uncertain significance,0.0,2.0,3) Uncertain significance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC_000962.3,5075,cg,cgtc,gyrB,upstream_gene_variant,Rv0005,c.-164_-163insTC,NaN,NaN
3,gyrB,Rv0005,gyrB_c-165t,NaN,5075,c,t,NaN,NaN,13,793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,423.0,5) Not assoc w R,4.0,370.0,5) Not assoc w R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC_000962.3,5075,c,t,gyrB,upstream_gene_variant,Rv0005,c.-165C>T,NaN,NaN
4,gyrB,Rv0005,gyrB_-165_ins_1_cg_cgt,NaN,5075,cg,cgt,NaN,NaN,0,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38.0,3) Uncertain significance,0.0,34.0,3) Uncertain significance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC_000962.3,5075,cg,cgt,gyrB,upstream_gene_variant,Rv0005,c.-164_-163insT,NaN,NaN


In [22]:
for (index, row) in data[:5].iterrows():
    garc = []
    #Pull out gene name, pos, ref and alt
    gene = row["gene_name"]
    genes.add(gene)
    if masks.get(gene) is None:
        #Cache the masks
        masks = {gene: parse.get_masks(reference, gene)}
    pos = str(row["final_annotation.Position"])#Cast to a str for str.split(',')
    ref = row["final_annotation.ReferenceNucleotide"]
    alt = row["final_annotation.AlternativeNucleotide"]

    #Check for multiple positions defined within pos
    if len(pos.split(",")) > 1:
        #There is more than 1 mutation detailed in this row, so skip it
        print("Mulitple muations per row: ", gene, pos, ref, alt, sep="\t")
        continue
    else:            
        garc += parse.to_garc(reference, gene, int(pos), ref, alt, masks)
        if len(garc) > 1:
            #There is more than 1 mutation generated from this row, so skip it
            print("Multiple mutations per row: ", gene, pos, ref, alt, garc, sep="\t")
            continue

5070 c t
5075 cg c
5075 cg cgtc
5075 c t
5075 cg cgt


In [17]:
masks

{'gyrB': (array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  array([False, False, False, ..., False, False, False]))}